In [1]:
import findspark
findspark.add_packages("graphframes:graphframes:0.8.0-spark3.0-s_2.12")
findspark.init()
import pyspark
import os
os.environ['PYSPARK_SUBMIT_ARGS']="--master local[2] pyspark-shell"
sc = pyspark.SparkContext(appName = "MyAPP")
from pyspark.sql.functions import *
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)
sc.addPyFile('/home/msbd5003/graphframes-0.8.0-spark3.0-s_2.12.jar')

/home/msbd5003/spark/python/pyspark/context.py:220: DeprecationWarning: Support for Python 2 and Python 3 prior to version 3.6 is deprecated as of Spark 3.0. See also the plan for dropping Python 2 support at https://spark.apache.org/news/plan-for-dropping-python-2-support.html.
  DeprecationWarning)


In [28]:
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import *
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import pandas as pd
import numpy as np
from datetime import date
from pyspark.sql.types import FloatType
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType


In [29]:
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [30]:
dataset1 = spark.read.csv('data/dataset1.csv',header=True, inferSchema=True)
dataset2 = spark.read.csv('data/dataset2.csv',header=True, inferSchema=True)
#dataset3 is for competition
#dataset3 = spark.read.csv('data/dataset3.csv',header=True, inferSchema=True)

print(dataset1.count())
print(dataset2.count())
print(dataset1.columns)
print(len(dataset1.columns))
print(len(dataset2.columns))

1994
5594
['User_id', 'Date_received', 'Discount_rate', 'Distance', 'Day_of_mouth', 'Days_distance', 'Total_sales', 'Sales_use_coupon', 'Total_coupon', 'Merchant_min_distance', 'Merchant_max_distance', 'Merchant_mean_distance', 'Merchant_median_distance', 'Merchant_coupon_transfer_rate', 'Coupon_rate', 'count_merchant', 'user_min_distance', 'user_max_distance', 'user_mean_distance', 'user_median_distance', 'buy_use_coupon', 'buy_total', 'coupon_received', 'avg_user_date_datereceived_gap', 'min_user_date_datereceived_gap', 'max_user_date_datereceived_gap', 'buy_use_coupon_rate', 'user_coupon_transfer_rate', 'User_merchant_buy_total', 'User_merchant_received', 'User_merchant_buy_use_coupon', 'User_merchant_any', 'User_merchant_buy_common', 'User_merchant_coupon_transfer_rate', 'User_merchant_coupon_buy_rate', 'User_merchant_coupon_rate', 'User_merchant_coupon_common_rate', 'Month_Receive_same_coupon_count', 'Month_Receive_all_coupon_count', 'Month_same_coupon_lastone', 'Month_same_coupon

In [31]:
minus_to_zero = udf(lambda x : 0 if x == -1 else x)

#for regression, label only 1 or 0
dataset1 = dataset1.withColumn('Label',minus_to_zero(col('Label'))) 
dataset2 = dataset2.withColumn('Label',minus_to_zero(col('Label'))) 
print(dataset1.count())
print(dataset2.count())

1994
5594


In [32]:


def cal_discount_rate(s):
    s =str(s)
    s = s.split(':')
    
    if len(s)==1:
        #print(s[0])
        return float(s[0])
    else:
        return float(1.0-float(s[1])/float(s[0]))
    
calc_discount_rate = udf(lambda x : cal_discount_rate(x))

check_not_float = udf(lambda x : x if type(x) == float else float(x))
    
dataset_train = dataset2.union(dataset1)
#dataset_train = dataset_train.replace
dataset_train = dataset_train.withColumn('Distance',col('Distance').cast(IntegerType()))
dataset_train.select('Discount_rate').show()
dataset_train = dataset_train.withColumn('Discount_rate',calc_discount_rate(col('Discount_rate')))
dataset_train = dataset_train.withColumn('Discount_rate',check_not_float(col('Discount_rate')))

from pyspark.sql.types import DoubleType
dataset_train = dataset_train.withColumn('Discount_rate',col('Discount_rate').cast(DoubleType()))
dataset_train = dataset_train.select([col(c).cast(DoubleType()) for c in dataset_train.columns])

dataset_train.select('Discount_rate').show()
dataset_train.printSchema()
print(dataset_train.count())
print(len(dataset_train.columns))



+-------------+
|Discount_rate|
+-------------+
|        20:05|
|    100:30:00|
|        20:01|
|        20:01|
|    100:10:00|
|     30:10:00|
|        20:01|
|        20:01|
|        20:01|
|        10:01|
|        20:05|
|          0.8|
|    100:30:00|
|        10:05|
|     30:05:00|
|        10:01|
|        20:05|
|     30:05:00|
|     50:05:00|
|        10:01|
+-------------+
only showing top 20 rows

+------------------+
|     Discount_rate|
+------------------+
|              0.75|
|               0.7|
|              0.95|
|              0.95|
|               0.9|
|0.6666666666666667|
|              0.95|
|              0.95|
|              0.95|
|               0.9|
|              0.75|
|               0.8|
|               0.7|
|               0.5|
|0.8333333333333334|
|               0.9|
|              0.75|
|0.8333333333333334|
|               0.9|
|               0.9|
+------------------+
only showing top 20 rows

root
 |-- User_id: double (nullable = true)
 |-- Date_receiv

In [33]:
drop_list = ['User_id']
dataset_train = dataset_train.drop(*drop_list)
print(dataset_train.count())
print(dataset_train.columns)

7588
['Date_received', 'Discount_rate', 'Distance', 'Day_of_mouth', 'Days_distance', 'Total_sales', 'Sales_use_coupon', 'Total_coupon', 'Merchant_min_distance', 'Merchant_max_distance', 'Merchant_mean_distance', 'Merchant_median_distance', 'Merchant_coupon_transfer_rate', 'Coupon_rate', 'count_merchant', 'user_min_distance', 'user_max_distance', 'user_mean_distance', 'user_median_distance', 'buy_use_coupon', 'buy_total', 'coupon_received', 'avg_user_date_datereceived_gap', 'min_user_date_datereceived_gap', 'max_user_date_datereceived_gap', 'buy_use_coupon_rate', 'user_coupon_transfer_rate', 'User_merchant_buy_total', 'User_merchant_received', 'User_merchant_buy_use_coupon', 'User_merchant_any', 'User_merchant_buy_common', 'User_merchant_coupon_transfer_rate', 'User_merchant_coupon_buy_rate', 'User_merchant_coupon_rate', 'User_merchant_coupon_common_rate', 'Label_user_merchant_coupon_count', 'Label_merchant_coupon_count', 'Coupon_count_later', 'Month_Receive_same_coupon_count', 'Month_R

In [34]:
col_list = ['Date_received', 'Discount_rate', 'Distance', 'Day_of_mouth', 'Days_distance', 'Total_sales', 'Sales_use_coupon', 'Total_coupon', 'Merchant_min_distance', 'Merchant_max_distance', 'Merchant_mean_distance', 'Merchant_median_distance', 'Merchant_coupon_transfer_rate', 'Coupon_rate', 'count_merchant', 'user_min_distance', 'user_max_distance', 'user_mean_distance', 'user_median_distance', 'buy_use_coupon', 'buy_total', 'coupon_received', 'avg_user_date_datereceived_gap', 'min_user_date_datereceived_gap', 'max_user_date_datereceived_gap', 'buy_use_coupon_rate', 'user_coupon_transfer_rate', 'User_merchant_buy_total', 'User_merchant_received', 'User_merchant_buy_use_coupon', 'User_merchant_any', 'User_merchant_buy_common', 'User_merchant_coupon_transfer_rate', 'User_merchant_coupon_buy_rate', 'User_merchant_coupon_rate', 'User_merchant_coupon_common_rate', 'Label_user_merchant_coupon_count', 'Label_merchant_coupon_count', 'Coupon_count_later', 'Month_Receive_same_coupon_count', 'Month_Receive_all_coupon_count', 'Month_same_coupon_lastone', 'Month_same_coupon_firstone', 'Day_Receive_all_coupon_count', 'Day_Receive_same_coupon_count', 'Is_weekend', 'Weekday1', 'Weekday2', 'Weekday3', 'Weekday4', 'Weekday5', 'Weekday6', 'Weekday7']
col_list_not_rate_distance = ['Date_received','Day_of_mouth', 'Days_distance', 'Total_sales', 'Sales_use_coupon', 'Total_coupon', 'Merchant_min_distance', 'Merchant_max_distance', 'Merchant_mean_distance', 'Merchant_median_distance', 'Merchant_coupon_transfer_rate', 'Coupon_rate', 'count_merchant', 'user_min_distance', 'user_max_distance', 'user_mean_distance', 'user_median_distance', 'buy_use_coupon', 'buy_total', 'coupon_received', 'avg_user_date_datereceived_gap', 'min_user_date_datereceived_gap', 'max_user_date_datereceived_gap', 'buy_use_coupon_rate', 'user_coupon_transfer_rate', 'User_merchant_buy_total', 'User_merchant_received', 'User_merchant_buy_use_coupon', 'User_merchant_any', 'User_merchant_buy_common', 'User_merchant_coupon_transfer_rate', 'User_merchant_coupon_buy_rate', 'User_merchant_coupon_rate', 'User_merchant_coupon_common_rate', 'Label_user_merchant_coupon_count', 'Label_merchant_coupon_count', 'Coupon_count_later', 'Month_Receive_same_coupon_count', 'Month_Receive_all_coupon_count', 'Month_same_coupon_lastone', 'Month_same_coupon_firstone', 'Day_Receive_all_coupon_count', 'Day_Receive_same_coupon_count', 'Is_weekend', 'Weekday1', 'Weekday2', 'Weekday3', 'Weekday4', 'Weekday5', 'Weekday6', 'Weekday7']

In [35]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=col_list, outputCol = 'features', handleInvalid='skip')
#assembler = VectorAssembler(inputCols=col_list, outputCol = 'features', handleInvalid='keep')
#assembler = VectorAssembler(inputCols=col_list_not_rate, outputCol = 'features')
#print(dataset_train.count())
dataset_train = assembler.transform(dataset_train)
dataset_train = dataset_train.drop(*col_list)
print(dataset_train.count())
dataset_train.printSchema()
dataset_train.show()
#dataset_train_ass.select('features').show(10,False)

6945
root
 |-- Label: double (nullable = true)
 |-- features: vector (nullable = true)

+-----+--------------------+
|Label|            features|
+-----+--------------------+
|  1.0|[2.0160521E7,0.75...|
|  0.0|[2.0160521E7,0.7,...|
|  0.0|[2.0160516E7,0.95...|
|  0.0|[2.016061E7,0.95,...|
|  0.0|[2.0160525E7,0.9,...|
|  0.0|[2.016053E7,0.666...|
|  1.0|[2.0160516E7,0.95...|
|  0.0|[2.016053E7,0.95,...|
|  0.0|[2.0160522E7,0.75...|
|  0.0|[2.0160613E7,0.8,...|
|  0.0|[2.0160525E7,0.7,...|
|  0.0|[2.0160525E7,0.5,...|
|  1.0|[2.0160602E7,0.83...|
|  0.0|[2.0160515E7,0.9,...|
|  0.0|[2.0160525E7,0.75...|
|  0.0|[2.0160615E7,0.83...|
|  1.0|[2.0160522E7,0.9,...|
|  1.0|[2.0160525E7,0.9,...|
|  0.0|[2.0160517E7,0.83...|
|  1.0|[2.0160525E7,0.83...|
+-----+--------------------+
only showing top 20 rows



In [36]:

#train validation test 0.8:0.1:0.1  seed 1
#rdd_dataset = dataset_train.randomSplit([0.8,0.1,0.1],1)
rdd_dataset = dataset_train.randomSplit([0.9,0.1],1)
#dataset3 is testing data set
#feature_list = rdd_dataset[0].columns
#feature_list.remove('Label')
#rdd_dataset[0] = rdd_dataset[0].withColumn('features', concat_ws(",",array(*feature_list))).drop(*feature_list)
#print(rdd_dataset[0].count())
#rdd_dataset[0].show(5)


In [37]:
lr = LogisticRegression(maxIter=1000, regParam=0.01, labelCol = 'Label')

In [38]:
rdd_dataset[0].printSchema()
model1 = lr.fit(rdd_dataset[0])

root
 |-- Label: double (nullable = true)
 |-- features: vector (nullable = true)



In [39]:
prediction_v = model1.transform(rdd_dataset[1])
prediction_v.printSchema()
numSuccesses = prediction_v.where('Label == prediction').count()


numInspections = prediction_v.count()

print ("There were %d inspections and there were %d successful predictions" % (numInspections, numSuccesses))
print("This is a %d%% success rate" % (float(numSuccesses) / float(numInspections) * 100))

#prediction_t = model1.transform(rdd_dataset[2])
#numSuccesses = prediction_t.where('Label == prediction').count()
#numInspections = prediction_t.count()
#print ("There were %d inspections and there were %d successful predictions" % (numInspections, numSuccesses))
#print("This is a %d%% success rate" % (float(numSuccesses) / float(numInspections) * 100))

root
 |-- Label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)

There were 688 inspections and there were 489 successful predictions
This is a 71% success rate
